## Imports

In [3]:
import pandas as pd
import numpy as np

## Data Input
Source : https://www.foodstandards.gov.au/science-data/monitoringnutrients/afcd/australian-food-composition-database-download-excel-files

In [4]:
data_path = "data//source//"

## Data Loading from Excel Files
In this section, we are loading two different datasets obtained from the Australian Food Composition Database using the Pandas library. 
We are loading it using `pd.read_excel` located in a directory path specified by the variable `data_path`.

1. **Food Dataset**: 
   - This dataset contains background information relating to each food.

2. **Nutrient Dataset**: 
   - This dataset contains the nutrient data available for each food, with the nutrient data provided in two ways where tab 1: Per 100 g – all foods and all beverages are reported per 100 g edible portion and tab 2: Per 100 mL – beverages and other liquid foods only, reported per 100 mL edible portion.

In [5]:
food = pd.read_excel(data_path + 'food_file.xlsx')
nutrient = pd.read_excel(data_path + 'nutrient_file.xlsx', sheet_name = 'All solids & liquids per 100g')

## Data Preprocessing 
We check for missing values across all columns in the `food` and then the `nutrient` dataset. 

### Output Interpretation:
From the output, we observe that most columns do not have missing values (`False`). However, columns like `Classification` and `Classification Name` do contain missing values (`True`). This insight will guide us in handling these missing values, which could involve imputing them, dropping them, or analyzing them further to understand the reasons behind their absence.

In [6]:
food.isna().any()

Public Food Key        False
Food Profile ID        False
Derivation             False
Food Name              False
Food Description       False
Sampling Details       False
Nitrogen Factor        False
Fat Factor             False
Specific Gravity       False
Analysed Portion       False
Unanalysed Portion     False
Classification          True
Classification Name     True
dtype: bool

## Quantifying Missing Values in `Classification` and `Classification Name` columns
This step helps in understanding the extent of missing data and informs subsequent data cleaning or imputation strategies.
The output is printed to give a clear count of how many missing values are present in each of these columns.

### Output Interpretation:
- The printed output indicates that there is only 1 missing value in both the `Classification` and `Classification Name` columns.

- **Matching Number of Missing Values**: The fact that both columns have exactly 1 missing value each suggests a possibility that the missing value in both columns comes from the same row. This could occur, for example, if a particular food item was not properly classified, resulting in missing entries in both related columns.

In [7]:
missing_values_sum_classification = food['Classification'].isna().sum()
missing_values_sum_classification_name = food['Classification Name'].isna().sum()

print("Sum of missing values in 'classification' column:", missing_values_sum_classification)
print("Sum of missing values in 'classification name' column:", missing_values_sum_classification_name)

Sum of missing values in 'classification' column: 1
Sum of missing values in 'classification name' column: 1


## Identifying Rows with Missing Values in the Food Dataset
In this section, we focus on pinpointing the exact rows in the `food` dataset that contain missing values. 

### Output Interpretation:
- The output shows `Index([1616], dtype='int64')`, indicating that row index 1616 in the pandas DataFrame contains missing values.
- Note that the pandas index 1616 corresponds to row number 1618 in the Excel file since pandas indexing starts at 0, whereas Excel's row numbering starts at 1. In addition, Excel often has a header row which pandas considers as part of the data.
- The row at index 1616, identified as having missing values, is actually just a repetition of the column headers, and can be safely removed from the dataset without losing any valuable data.

In [8]:
nan_row_indices = food[food.isna().any(axis=1)].index
print(nan_row_indices)

Index([1616], dtype='int64')


## Removing Duplicate Header Row from the DataFrame
- We drop the row to ensure the integrity and cleanliness of our dataset.

In [9]:
food = food.drop(1616)

## Removing unnecessary columns for Macro Food Calculator Database
We retain only the most relevant fields, such as the unique code Public Food Key, food name, and detailed description.

In [10]:
food.drop(['Derivation', 'Food Profile ID', 'Sampling Details', 'Specific Gravity', 'Analysed Portion', 'Unanalysed Portion', 'Nitrogen Factor', 'Fat Factor'], axis=1, inplace=True, errors = 'ignore')

# Identifying Missing Values in `Nutrient` dataset
## Output Interpretation
- Columns such as `Classification`, `Serine (mg)`, `Threonine (mg)`, `Tyrosine (mg)`, and `Valine (mg)` show `True`, which tell us that these nutrients have missing data points for some food items.

In [11]:
nutrient.isna().any()

Public Food Key                                  False
Classification                                    True
Food Name                                        False
Energy with dietary fibre, equated \n(kJ)        False
Energy, without dietary fibre, equated \n(kJ)    False
                                                 ...  
Serine \n(mg)                                     True
Threonine \n(mg)                                  True
Tyrosine \n(mg)                                   True
Tryptophan \n(mg)                                False
Valine \n(mg)                                     True
Length: 293, dtype: bool

## Locating Rows with Missing 'Classification' in the Nutrient Dataset
- The code below filters out rows where the `Classification` column is `NaN` and retrieve their indices.

### Output Interpretation:
- These indices correspond to rows 126 and 132 in the pandas DataFrame. 
- As we investigate the rows with missing data, we notice that 

In [12]:
nan_indices = nutrient[nutrient['Classification'].isna()].index
print(nan_indices) 

Index([126, 132], dtype='int64')


## Filling Missing 'Classification' Values in the Nutrient Dataset Using Data from the Food Dataset
- First, we create `classification_map`, a Series from the `food` DataFrame. In this Series, 'Public Food Key' is set as the index, and 'Classification' is the value. This mapping provides a reference to look up the 'Classification' of each food item based on its unique key.
- Next, we use this mapping to fill in missing 'Classification' values in the `nutrient` DataFrame. 

In [13]:
# Mapping Series from the 'food' DataFrame where 'Public Food Key' is the index and 'Classification' is the value
classification_map = food.set_index('Public Food Key')['Classification']

# Use the map to fill NaN values in the 'Classification' column of 'nutrient' DataFrame
nutrient['Classification'] = nutrient['Classification'].fillna(nutrient['Public Food Key'].map(classification_map))

## Removing unnecessary columns for Macro Food Calculator Database

In [14]:
# Drop unncessary columns from the 'nutrient' df
columns_to_keep = [
    'Public Food Key', 'Classification', 'Food Name', 
    'Energy with dietary fibre, equated \n(kJ)',
    'Protein \n(g)', 'Fat, total \n(g)',
    'Total dietary fibre \n(g)', 'Total sugars (g)', 
    'Added sugars (g)', 'Free sugars \n(g)', 'Starch \n(g)',
    'Available carbohydrate, with sugar alcohols \n(g)',
    'Caffeine \n(mg)'
]

# Index the DataFrame with this list to keep only these columns
nutrient = nutrient[columns_to_keep]

## Dataframe Merging 
**Setting Index**:
- We prepare `nutrient` and `food` DataFrames for merging by setting their 'Public Food Key' columns as indexes.

In [15]:
# Setting 'Public Food Key' as index for original dfs
nutrient.set_index('Public Food Key', inplace=True)
food.set_index('Public Food Key', inplace=True)

## Checking for Column Consistency Across `nutrient` and `food` DataFrames

**Identifying Common Keys**:
  - Finding common indices from `Public Food Key` in both DataFrames using `intersection`. 

### Output Interpretation:
- The output `False` for the 'Classification' column indicates discrepancies between the two datasets. This suggests that for some common food items, the classification differs between the `nutrient` and `food` DataFrames.


In [16]:
# Find the common keys in both dataframes
common_keys = nutrient.index.intersection(food.index)

# Iterate through each column to compare
for column in ['Food Name', 'Classification']:
    # Compare the columns for common keys and check if all matching
    matching_columns = nutrient.loc[common_keys, column].equals(food.loc[common_keys, column])
    print(f"{column} columns identical for common primary keys in both dataframes?", matching_columns)

Food Name columns identical for common primary keys in both dataframes? True
Classification columns identical for common primary keys in both dataframes? False


## Investigating Discrepancies in 'Classification' Column Across DataFrames
Since the previous code identified a discrepancy in the 'Classification' column between the `nutrient` and `food` DataFrames, we further investigate to pinpoint the exact mismatches.

### Output Interpretation:
- We find a mismatch for the key `F009802`, indicating that the classification for this particular food item is different in the `nutrient` DataFrame compared to the `food` DataFrame.

In [17]:
# Find mismatches in the current column
mismatch_mask = nutrient.loc[common_keys, 'Classification'] != food.loc[common_keys, 'Classification']
mismatched_keys = mismatch_mask[mismatch_mask].index.tolist()

# Print the keys where mismatches were found
if mismatched_keys:  # If the list is not empty
        print(f"Mismatches in '{column}' for keys:", mismatched_keys)
else:
        print(f"No mismatches found in '{column}' column.")

Mismatches in 'Classification' for keys: ['F009802']


## Resolving the Classification Discrepancy for Key 'F009802'

**Accessing Classification Data**:
  - We access the 'Classification' data for the key 'F009802' in both `food` and `nutrient` DataFrames using the `loc` method to directly compare the classification values assigned in each dataset.
 
### Output Interpretation:
- The output reveals that the classification codes for this particular food item are different in the two datasets, with `food` having a classification of `28202.0` and `nutrient` having `28301.0`.

In [18]:
# Accessing data by the index 'F009802'
food_classification = food.loc['F009802', 'Classification']
print(food_classification)

nutrient_classification = nutrient.loc['F009802', 'Classification']
print(nutrient_classification)

28202.0
28301.0


## Updating Classification for Key 'F009802'
We have chosen to update the classification in the `food` DataFrame to match that in the `nutrient` DataFrame.

**Rationale for Choosing '28301.0'**:
  - The key 'F009802' corresponds to a 'Bar, snack, with >35% nuts, added protein'. In the `nutrient` DataFrame, this item is classified as '28301.0', while in the `food` DataFrame, it is classified as '28202.0'.
  - Upon closer examination, we found that another item, 'F009801' (Bar, snack, with >55% nuts), is consistently classified as '28202' in both dataframes. This consistency suggests that '28202' might represent a general category for nut-based snack bars.
  - However, since 'F009802' includes added protein, which could differentiate it nutritionally and categorically, the classification '28301.0' in the `nutrient` DataFrame may more accurately reflect its unique characteristics. Therefore, we choose to align the `food` DataFrame with this classification.


In [19]:
# Updating the 'Classification' value for the row with 'Public Food Key' of 'F009802'
food.loc['F009802', 'Classification'] = '28301.0'

/var/folders/ww/sdpjdv8n5bxg6ylcbh7qm0vc0000gn/T/ipykernel_73969/1849043427.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '28301.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  food.loc['F009802', 'Classification'] = '28301.0'


## Verifying the Alignment of 'Classification' Columns Across Both DataFrames
We perform a final check to ensure alignment and consistency across the `food` and `nutrient` DataFrames.

### Output Interpretation:
- The output indicates that there are still some mismatches in the 'Classification' column between the food and nutrient DataFrames for certain common items.

In [22]:
# Compare 'Classification' columns for common keys
classification_match = (food.loc[common_keys, 'Classification'] == nutrient.loc[common_keys, 'Classification']).all()
print("Do 'Classification' columns match for all common 'Public Food Key'?", classification_match)

Do 'Classification' columns match for all common 'Public Food Key'? True


## Converting 'Classification' Column to String Type for Consistency
- We convert both columns as strings, eliminating data type mismatches.
- If we run the code above once again, the mismatch in the `Clasffication` column will be resolved. 

In [21]:
nutrient['Classification'] = nutrient['Classification'].astype(str)
food['Classification'] = food['Classification'].astype(str)